In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time

### Heuristic simplification
- When simplifying ZX-diagrams with T-spiders, simplification routines like full_reduce lead to a very high two-qubit gate count.
- When using heuristic-based approaches we can circumvent the problem to some extent leading to better overall circuit cost after optimization

In [2]:
random.seed(1344)
g = zx.generate.cliffordT(qubits=8, depth=1000, p_t=0.3)
c = zx.Circuit.from_graph(g)
print(c.stats())

Circuit  on 8 qubits with 1000 gates.
        292 is the T-count
        708 Cliffords among which 
        230 2-qubit gates (230 CNOT, 0 other) and
        0 Hadamard gates.


In [3]:
c = zx.optimize.basic_optimization(c.split_phase_gates())
g = c.to_graph()
g_tele = zx.simplify.teleport_reduce(g)
g_tele.track_phases = False
zx.Circuit.from_graph(g).split_phase_gates().stats()

'Circuit  on 8 qubits with 652 gates.\n        122 is the T-count\n        530 Cliffords among which \n        219 2-qubit gates (208 CNOT, 11 other) and\n        133 Hadamard gates.'

In [19]:
g_full = g_tele.copy()
zx.simplify.full_reduce(g_full)
zx.extract_circuit(g_full.copy()).stats()

'Circuit  on 8 qubits with 381 gates.\n        69 is the T-count\n        312 Cliffords among which \n        214 2-qubit gates (34 CNOT, 180 other) and\n        93 Hadamard gates.'

In [8]:
g_greedy = g_tele.copy()
zx.simplify.greedy_simp(g_greedy)
zx.extract_circuit(g_greedy.copy()).stats()

'Circuit  on 8 qubits with 723 gates.\n        122 is the T-count\n        601 Cliffords among which \n        215 2-qubit gates (5 CNOT, 210 other) and\n        292 Hadamard gates.'

In [4]:
g_nu = g_tele.copy()
zx.simplify.greedy_simp_neighbors(g_nu)
zx.extract_circuit(g_nu.copy()).stats()

apply pivot match  (5, (819, 820), None, None)
apply pivot match  (5, (807, 808), None, None)
apply pivot match  (5, (813, 814), None, None)
apply pivot match  (5, (795, 796), None, None)
apply pivot match  (5, (801, 802), None, None)
apply pivot match  (5, (783, 784), None, None)
apply pivot match  (5, (789, 790), None, None)
apply pivot match  (3, (140, 141), None, None)
apply pivot match  (3, (313, 316), None, None)
apply pivot match  (3, (589, 620), None, None)
apply pivot match  (2, (642, 653), None, None)
apply pivot match  (2, (815, 818), None, None)
apply pivot match  (2, (302, 323), None, None)
apply pivot match  (2, (526, 588), None, None)
apply pivot match  (2, (785, 806), None, None)
apply pivot match  (2, (777, 797), None, None)
apply pivot match  (1, (202, 224), None, None)
apply pivot match  (1, (159, 164), 156, None)
unfuse to neighbor  159 156 832 831
apply pivot match  (1, (591, 598), None, None)
apply pivot match  (1, (325, 326), None, None)
apply pivot match  (1, (3

'Circuit  on 8 qubits with 724 gates.\n        122 is the T-count\n        602 Cliffords among which \n        214 2-qubit gates (3 CNOT, 211 other) and\n        295 Hadamard gates.'